In [ ]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import numpy as np
import pandas as pd
import pickle 
import os
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor

# local import
from make_datasets import make_data
import models

In [ ]:
from shapely import wkt
#retrieve cleaned data frames 
data_dir = '/Users/jyontika/Desktop/opioid-overdose-models/cook-county/cleaning-cook-county/'
gdf_annual = pd.read_csv(f'{data_dir}/cook_county_gdf_year.csv')

#convert to gpd (was having trouble importing csv as gdf)
gdf_annual['geometry'] = gdf_annual['geometry'].apply(wkt.loads)
gdf_annual = gpd.GeoDataFrame(gdf_annual, geometry='geometry')
gdf_annual.crs = {'init': 'EPSG:4269'}
type(gdf_annual)

data_gdf = gdf_annual

Process dataframe into a data frame with a Multiindex on location and time

In [ ]:
data_gdf.shape

In [ ]:
# Name the important columns
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

# These are the columns we could possibly want in the X dataframe
x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']

# These are the columns we could want in the Y dataframe
y_idx_cols = [geography_col, timestep_col, outcome_col]

# These are the features we want
features_only = ['deaths']
add_spacetime = True
add_svi = True
if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


# #jyontika's parameters
validation_year = 2020
first_test_year = 2021
last_test_year = 2022
first_test_timestep = 6
last_test_timestep = 7
lookback_years= 3 #use 3 lookback years
first_train_eval_year = validation_year - lookback_years #2017
last_train_eval_year = validation_year -1 #2019

In [ ]:
# Create the multiindex
multiindexed_gdf = data_gdf.set_index([geography_col, timestep_col])

# re-add the timestep column as a feature because it's useful
multiindexed_gdf[timestep_col] = multiindexed_gdf.index.get_level_values(timestep_col)

# Track number of locations
num_geoids = len(data_gdf[geography_col].unique())

In [ ]:
print(lookback_years)
print(num_geoids)
print(len(features_only))

In [ ]:
x_BSF, y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          features_only, num_geoids)
x_test_BSF, y_test_BS = make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          features_only, num_geoids)
          
# For the weighted historical average model, we only use deaths as features
x_BSF_death_only, y_BS_death_only = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          ['deaths'], num_geoids)
x_test_BSF_death_only, y_test_BS_death_only =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          ['deaths'], num_geoids)


In [ ]:
x_BSF.shape

In [ ]:
y_BS.shape

### All Zeroes Model
### lookback = 3 (param)

In [ ]:
bpr_over_time_zeroes = models.all_zeroes_model(multiindexed_gdf,
                                        first_test_timestep, last_test_timestep,
                                        num_geoids, bpr_uncertainty_samples=15)

In [ ]:
print(f"2021 Average: {np.mean(bpr_over_time_zeroes[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_zeroes[0]) + \
                          np.array(bpr_over_time_zeroes[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


In [ ]:

# Assuming multiindexed_gdf contains the actual target predictions with 'year' and 'deaths' columns
actual_values_2021 = multiindexed_gdf[multiindexed_gdf['year'] == 2021]['deaths'].values
actual_values_2022 = multiindexed_gdf[multiindexed_gdf['year'] == 2022]['deaths'].values

# Calculate the model predictions 
bpr_samples_2021 = np.mean(bpr_over_time_zeroes[0])
bpr_samples_2022 = np.mean(bpr_over_time_zeroes[1])

# Calculate RMSE for model
rmse_2021 = np.sqrt(np.mean((bpr_samples_2021 - actual_values_2021)**2))
rmse_2022 = np.sqrt(np.mean((bpr_samples_2022 - actual_values_2022)**2))

joint_rmse = np.mean([rmse_2021, rmse_2022])

print(f"Zeroes model RMSE for 2021: {rmse_2021:.2f}")
print(f"Zeroes model RMSE for 2022: {rmse_2022:.2f}")
print(f"Joint RMSE for 2021 and 2022: {joint_rmse:.2f}")
print(" ")

mae_2021_samples = np.mean(np.abs(bpr_samples_2021 - actual_values_2021))
mae_2022_samples = np.mean(np.abs(bpr_samples_2022 - actual_values_2022))

# Take the average MAE for each year
mae_2021 = np.mean(mae_2021_samples)
mae_2022 = np.mean(mae_2022_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2021, mae_2022])

print(f"Zeroes model MAE for 2021: {mae_2021:.2f}")
print(f"Zeroes model MAE for 2022: {mae_2022:.2f}")
print(f"Joint MAE for 2021 and 2022: {joint_mae:.2f}")

### Last Year
#### lookback = 1

In [ ]:
bpr_over_time_last_time = models.last_time_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1,bpr_uncertainty_samples=15)

In [ ]:
print(f"2021 Average: {np.mean(bpr_over_time_last_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_last_time[0]) + \
                          np.array(bpr_over_time_last_time[1]))/2
                        
print(f"""Last Year model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


In [ ]:
actual_values_2021 = multiindexed_gdf[multiindexed_gdf['year'] == 2021]['deaths'].values
actual_values_2022 = multiindexed_gdf[multiindexed_gdf['year'] == 2022]['deaths'].values

# Calculate the model predictions 
bpr_samples_2021 = np.mean(bpr_over_time_last_time[0])
bpr_samples_2022 = np.mean(bpr_over_time_last_time[1])

# Calculate RMSE for model
rmse_2021 = np.sqrt(np.mean((bpr_samples_2021 - actual_values_2021)**2))
rmse_2022 = np.sqrt(np.mean((bpr_samples_2022 - actual_values_2022)**2))

joint_rmse = np.mean([rmse_2021, rmse_2022])

print(f"Last Year model RMSE for 2021: {rmse_2021:.2f}")
print(f"Last Year model RMSE for 2022: {rmse_2022:.2f}")
print(f"Joint RMSE for 2021 and 2022: {joint_rmse:.2f}")
print(" ")

mae_2021_samples = np.mean(np.abs(bpr_samples_2021 - actual_values_2021))
mae_2022_samples = np.mean(np.abs(bpr_samples_2022 - actual_values_2022))

# Take the average MAE for each year
mae_2021 = np.mean(mae_2021_samples)
mae_2022 = np.mean(mae_2022_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2021, mae_2022])

print(f"Last Year model model MAE for 2021: {mae_2021:.2f}")
print(f"Last Year model MAE for 2022: {mae_2022:.2f}")
print(f"Joint MAE for 2021 and 2022: {joint_mae:.2f}")


### Historical Average 
#### lookback = 6 years for cook 

In [ ]:
bpr_over_time_avg_time = models.historical_average_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1, 6, bpr_uncertainty_samples=15)

In [ ]:
print(f"2021 Average: {np.mean(bpr_over_time_avg_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_avg_time[0]) + \
                          np.array(bpr_over_time_avg_time[1]))/2
                        
print(f"""Historical Average model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


In [ ]:
actual_values_2021 = multiindexed_gdf[multiindexed_gdf['year'] == 2021]['deaths'].values
actual_values_2022 = multiindexed_gdf[multiindexed_gdf['year'] == 2022]['deaths'].values

# Calculate the model predictions 
bpr_samples_2021 = np.mean(bpr_over_time_avg_time[0])
bpr_samples_2022 = np.mean(bpr_over_time_avg_time[1])

# Calculate RMSE for model
rmse_2021 = np.sqrt(np.mean((bpr_samples_2021 - actual_values_2021)**2))
rmse_2022 = np.sqrt(np.mean((bpr_samples_2022 - actual_values_2022)**2))

joint_rmse = np.mean([rmse_2021, rmse_2022])

print(f"Historical Average RMSE for 2021: {rmse_2021:.2f}")
print(f"Historical Average RMSE for 2022: {rmse_2022:.2f}")
print(f"Joint RMSE for 2021 and 2022: {joint_rmse:.2f}")
print(" ")

mae_2021_samples = np.mean(np.abs(bpr_samples_2021 - actual_values_2021))
mae_2022_samples = np.mean(np.abs(bpr_samples_2022 - actual_values_2022))

# Take the average MAE for each year
mae_2021 = np.mean(mae_2021_samples)
mae_2022 = np.mean(mae_2022_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2021, mae_2022])

print(f"Historical Average  MAE for 2021: {mae_2021:.2f}")
print(f"Historical Average  MAE for 2022: {mae_2022:.2f}")
print(f"Joint MAE for 2021 and 2022: {joint_mae:.2f}")


### Weighted Historical Average
#### lookback = 6 years (for cook county)

In [ ]:
# Identical models, features are only difference
linear_poisson_weighted_avg = sklearn.linear_model.PoissonRegressor()
linear_poisson = sklearn.linear_model.PoissonRegressor()

# Params selected via grid search on validation. Need to re-do grid search for chicago
hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=3, max_leaf_nodes=2,
                                               l2_regularization=1, min_samples_leaf=100 )

In [ ]:
bpr_over_time_weight_avg = models.scikit_model(multiindexed_gdf, x_BSF_death_only,
                                               y_BS_death_only, x_test_BSF_death_only,
                                               linear_poisson_weighted_avg,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [ ]:
print(f"2021 Average: {np.mean(bpr_over_time_weight_avg[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_weight_avg[0]) + \
                          np.array(bpr_over_time_weight_avg[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


In [ ]:
actual_values_2021 = multiindexed_gdf[multiindexed_gdf['year'] == 2021]['deaths'].values
actual_values_2022 = multiindexed_gdf[multiindexed_gdf['year'] == 2022]['deaths'].values

# Calculate the model predictions 
bpr_samples_2021 = np.mean(bpr_over_time_weight_avg[0])
bpr_samples_2022 = np.mean(bpr_over_time_weight_avg[1])

# Calculate RMSE for model
rmse_2021 = np.sqrt(np.mean((bpr_samples_2021 - actual_values_2021)**2))
rmse_2022 = np.sqrt(np.mean((bpr_samples_2022 - actual_values_2022)**2))

joint_rmse = np.mean([rmse_2021, rmse_2022])

print(f"Weighted Historical Average RMSE for 2021: {rmse_2021:.2f}")
print(f"Weighted Historical Average RMSE for 2022: {rmse_2022:.2f}")
print(f"Joint RMSE for 2021 and 2022: {joint_rmse:.2f}")
print(" ")

mae_2021_samples = np.mean(np.abs(bpr_samples_2021 - actual_values_2021))
mae_2022_samples = np.mean(np.abs(bpr_samples_2022 - actual_values_2022))

# Take the average MAE for each year
mae_2021 = np.mean(mae_2021_samples)
mae_2022 = np.mean(mae_2022_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2021, mae_2022])

print(f"Weighted Historical Average  MAE for 2021: {mae_2021:.2f}")
print(f"Weighted Historical Average  MAE for 2022: {mae_2022:.2f}")
print(f"Joint MAE for 2021 and 2022: {joint_mae:.2f}")


### Linear (Poisson GLM baseline)
#### lookback years = 3

In [ ]:
bpr_over_time_linear = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               linear_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [ ]:
print(f"2021 Average: {np.mean(bpr_over_time_linear[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_linear[0]) + \
                          np.array(bpr_over_time_linear[1]))/2
                        
print(f"""Poisson GLM model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


In [ ]:
actual_values_2021 = multiindexed_gdf[multiindexed_gdf['year'] == 2021]['deaths'].values
actual_values_2022 = multiindexed_gdf[multiindexed_gdf['year'] == 2022]['deaths'].values

# Calculate the model predictions 
bpr_samples_2021 = np.mean(bpr_over_time_linear[0])
bpr_samples_2022 = np.mean(bpr_over_time_linear[1])

# Calculate RMSE for model
rmse_2021 = np.sqrt(np.mean((bpr_samples_2021 - actual_values_2021)**2))
rmse_2022 = np.sqrt(np.mean((bpr_samples_2022 - actual_values_2022)**2))

joint_rmse = np.mean([rmse_2021, rmse_2022])

print(f"Linear (Poisson GLM baseline) for 2021: {rmse_2021:.2f}")
print(f"Linear (Poisson GLM baseline) RMSE for 2022: {rmse_2022:.2f}")
print(f"Joint RMSE for 2021 and 2022: {joint_rmse:.2f}")
print(" ")

mae_2021_samples = np.mean(np.abs(bpr_samples_2021 - actual_values_2021))
mae_2022_samples = np.mean(np.abs(bpr_samples_2022 - actual_values_2022))

# Take the average MAE for each year
mae_2021 = np.mean(mae_2021_samples)
mae_2022 = np.mean(mae_2022_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2021, mae_2022])

print(f"Linear (Poisson GLM baseline)  MAE for 2021: {mae_2021:.2f}")
print(f"Linear (Poisson GLM baseline)  MAE for 2022: {mae_2022:.2f}")
print(f"Joint MAE for 2021 and 2022: {joint_mae:.2f}")


### Gradient Boosted Trees (Poisson)
#### lookback years = 3 

In [ ]:
bpr_over_time_tree = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               hist_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [ ]:
print(f"2021 Average: {np.mean(bpr_over_time_tree[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_tree[0]) + \
                          np.array(bpr_over_time_tree[1]))/2
                        
print(f"""Gradient Boosted Trees (Poisson) (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


In [ ]:
actual_values_2021 = multiindexed_gdf[multiindexed_gdf['year'] == 2021]['deaths'].values
actual_values_2022 = multiindexed_gdf[multiindexed_gdf['year'] == 2022]['deaths'].values

# Calculate the model predictions 
bpr_samples_2021 = np.mean(bpr_over_time_tree[0])
bpr_samples_2022 = np.mean(bpr_over_time_tree[1])

# Calculate RMSE for model
rmse_2021 = np.sqrt(np.mean((bpr_samples_2021 - actual_values_2021)**2))
rmse_2022 = np.sqrt(np.mean((bpr_samples_2022 - actual_values_2022)**2))

joint_rmse = np.mean([rmse_2021, rmse_2022])

print(f"Gradient Boosted Trees (Poisson) RMSE for 2021: {rmse_2021:.2f}")
print(f"Gradient Boosted Trees (Poisson) RMSE for 2022: {rmse_2022:.2f}")
print(f"Joint RMSE for 2021 and 2022: {joint_rmse:.2f}")
print(" ")

mae_2021_samples = np.mean(np.abs(bpr_samples_2021 - actual_values_2021))
mae_2022_samples = np.mean(np.abs(bpr_samples_2022 - actual_values_2022))

# Take the average MAE for each year
mae_2021 = np.mean(mae_2021_samples)
mae_2022 = np.mean(mae_2022_samples)

# Calculate the joint MAE (mean of individual MAE values from both years)
joint_mae = np.mean([mae_2021, mae_2022])

print(f"Gradient Boosted Trees (Poisson)  MAE for 2021: {mae_2021:.2f}")
print(f"Gradient Boosted Trees (Poisson)  MAE for 2022: {mae_2022:.2f}")
print(f"Joint MAE for 2021 and 2022: {joint_mae:.2f}")


### CASTNet
#### 3 lookback years

In [ ]:
#import CASTNet 
import sys
my_data ='/Users/jyontika/Desktop/opioid-overdose-models/CASTNet/'
CASTNet_path = os.path.join(my_data, 'hughes-CASTNet')
sys.path.append(CASTNet_path)


In [ ]:
import hughes_castnet_main as CASTNet
CASTNet.results['preds']

In [ ]:
all_predictions = results['preds']
prediction_matrix = np.array(all_predictions)
prediction_matrix = np.maximum(prediction_matrix, 0) #turn all negative values to 0

# Call the castnet_model function to calculate BPR for CASTNet predictions
bpr_results_castnet = models.castnet_model(prediction_matrix, true_values, num_locations=1328)

In [ ]:
print(f"2021 Average: {np.mean(bpr_results_castnet[0])}")

bpr_samples_both_years = (np.array(bpr_results_castnet[0]) + \
                          np.array(bpr_results_castnet[1]))/2
                        
print(f"""CASTNet model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")